In [52]:
#Import modules
import torch
import pandas as pd
import os
import numpy as np 
import matplotlib.pyplot as plt
from torchvision import transforms
from PIL import Image

device = ("cuda" if torch.cuda.is_available() else "cpu")

In [73]:
transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))
])

def load_and_transform(img_name):
    image = Image.open(img_name).convert("RGB")
    return transforms(image)

def imshow(img_path):
    image = Image.open(img_path).convert("RGB")
    image_tensor = transforms(image)

    img = image_tensor.numpy()
    img = np.transpose(img, (1,2,0))
    img = img * 0.5 + 0.5

    plt.imshow(img)
    plt.axis("off")
    plt.show()

In [ ]:
root_dir = 'image/'
class_folders = sorted(os.listdir(root_dir))

label_map = {folder: idx for idx, folder in enumerate(class_folders)}
del label_map['.DS_Store']

data = []

for folder in class_folders:
    folder_path = os.path.join(root_dir, folder)
    if os.path.isdir(folder_path):
        for img_name in os.listdir(folder_path):
            if img_name.endswith('.jpg'):
                data.append({"img_name": folder_path+'/'+img_name, "label": label_map[folder]})

df = pd.DataFrame(data)

df["label"] = df["label"].astype(int) 
df["label"] = df["label"] - 1  
df["img_tensors"] = df.apply(lambda row: load_and_transform(row["img_name"]), axis=1)

In [75]:
df.head(3)

,img_name,label,img_tensors
0,image/Normal/Normal_55.jpg,0,"[[[tensor(-1.), tensor(-1.), tensor(-1.), tens..."
1,image/Normal/Normal_41.jpg,0,"[[[tensor(-1.), tensor(-1.), tensor(-1.), tens..."
2,image/Normal/Normal_128.jpg,0,"[[[tensor(-1.), tensor(-1.), tensor(-1.), tens..."


In [77]:
from torch.utils.data import Dataset, DataLoader, random_split

class ImageDataset(Dataset):
    def __init__(self, df):
        self.img_tensors = df["img_tensors"].tolist()
        self.labels = df["label"].tolist()
    def __len__(self):
        return len(self.img_tensors)
    def __getitem__(self, idx):
        return self.img_tensors[idx], self.labels[idx]
    
dataset = ImageDataset(df)

train_size = int(0.8*len(dataset))
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

images, labels = next(iter(train_loader))
print(images.shape, labels.shape)

torch.Size([32, 3, 256, 256]) torch.Size([32])


In [ ]:
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F



class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)  
        self.pool = nn.MaxPool2d(2, 2)   
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool = nn.MaxPool2d(2, 2)  

        self.flatten_size = 16 * 61 * 61  
        self.fc1 = nn.Linear(self.flatten_size, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 5) 

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x))) 
        x = self.pool(F.relu(self.conv2(x))) 

        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(2):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data

        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 200 == 199:
            print(f'[Epoch {epoch+1}, Batch {i+1}] Loss: {running_loss / 200:.3f}')
            running_loss = 0.0

print("Training Finished")

correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy {100 * correct / total:.2f}%')

Training Finished
Accuracy 33.18%


In [ ]:
# Now we have data loaded, few things to do from here:
# - feature engineering
# - try out image preprocessing
# - augmenting dataset 
# - transformer architecture